# **LAB 01-Feed Forward & Back-Propagation Learning Algorithm**

### **20th January 2025**
# **REDDIPALLI SAI CHARISH**
# **CS22B1095**


---



---



---







#**QUESTION-1**
Implement the Perceptron algorithm from scratch in Python.
*   Initialize the weights with [0 0 0] and a learning rate of 0.0001.
*   For each iteration, calculate the output of the Perceptron for each input in the training set.
*  Use MSE to computer the error for all samples
* Update the weights using the gradient descent procedure.
*   Repeat the above steps until the Perceptron converges or a maximum number of iterations is reached.
* Test the trained Perceptron on a separate test set, explain how you came up with the test set.
* Use the step function as an  activation function in the output layer
Use the IRIS Dataset for the above, considering all four features: sepal length, sepal width, petal length, and petal width, but only two classes -  Setosa, and Versicolor.  Drop the feature vectors of the other class.


In [71]:
import numpy as np
import pandas as pd

## **Perceptron**

In [72]:
class Perceptron:
  def __init__(self,lr=0.0001, epochs=100):
    self.lr = lr
    self.epochs = epochs
    self.acv_func = self.step_func
    self.w = None

  def step_func(self, x):
    return np.where(x >= 0, 1, 0)

  def fit(self,X,y):
    X=np.insert(X,0,1,axis=1)
    n_samples,n_features = X.shape
    self.w = np.zeros(n_features)

    while self.epochs > 0:
      flag = 0
      for i,x_i in enumerate(X):
        dot_output = np.dot(x_i,self.w)
        y_pred = self.acv_func(dot_output)
        if y_pred != y[i]:
          self.w += self.lr * (y[i] - y_pred) * x_i
          flag = 1
      if flag == 0:
        break
      self.epochs -= 1

  def predict(self,X):
    X=np.insert(X,0,1,axis=1)
    return self.acv_func(np.dot(X,self.w))


In [73]:

iris_df = pd.read_csv("iris.csv").drop("Id", axis=1)
iris_df = iris_df[iris_df['Species'].isin(['Iris-setosa', 'Iris-versicolor'])]
print(f"Count: {len(iris_df)}")
iris_df

Count: 100


,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
95,5.7,3.0,4.2,1.2,Iris-versicolor
96,5.7,2.9,4.2,1.3,Iris-versicolor
97,6.2,2.9,4.3,1.3,Iris-versicolor
98,5.1,2.5,3.0,1.1,Iris-versicolor


In [74]:

X = iris_df.drop("Species", axis=1).values
y = iris_df['Species'].map({'Iris-setosa': 0, 'Iris-versicolor': 1}).values

In [75]:

def train_test_split(X, y, test_size=0.2):
    n_classes = len(np.unique(y))
    X_train, X_test, y_train, y_test = [], [], [], []
    for i in range(n_classes):
        X_class = X[y == i]
        y_class = y[y == i]
        n_samples = len(X_class)
        n_train = int(n_samples * (1 - test_size))
        X_train.extend(X_class[:n_train])
        X_test.extend(X_class[n_train:])
        y_train.extend(y_class[:n_train])
        y_test.extend(y_class[n_train:])
    return np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print("Test dataset for each class:")
print(f"{np.asarray(np.unique(y_test, return_counts=True)).T}")



Test dataset for each class:
[[ 0 10]
 [ 1 10]]


In [76]:
perceptron=Perceptron(lr=0.0001,epochs=100)
perceptron.fit(X_train,y_train)

In [77]:
y_pred=perceptron.predict(X_test)
result=pd.DataFrame({'Actual':y_test,'Predicted':y_pred})
result


,Actual,Predicted
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


In [78]:
accuracy=np.mean(y_pred==y_test)
accuracy

1.0